In [1]:
import functools
import time

import keras
import keras.backend as K
import matplotlib.pyplot as plt
# import scikitplot as skplt
import numpy as np
import pandas as pd
import scikitplot as skplt
import seaborn as sns
import tensorflow as tf
import xgboost as xgb
from keras.layers import Dense, Dropout, Input, Lambda
from keras.models import Model, Sequential, load_model
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, accuracy_score, auc, auc, average_precision_score, average_precision_score, \
    confusion_matrix, confusion_matrix, pairwise_distances, precision_score, precision_score, recall_score, \
    recall_score, roc_auc_score, roc_auc_score, roc_curve, roc_curve
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors
from rdkit.Chem import AllChem
from data_analysis import calculate_metrics, get_rdkit_features
import dill
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
def get_keras_simple_nn(num_units=35, activation='relu', drop_rate=0.2):
    model = Sequential()
    model.add(Dense(num_units, activation=activation))
    model.add(Dropout(drop_rate))
    model.add(Dense(num_units, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


baselines_map = {
    'knn_clf': KNeighborsClassifier(n_neighbors=1, weights='distance'),
    'random_forest': RandomForestClassifier(),
    'logistic_regression': LogisticRegression(),
    'svc': SVC(gamma='auto'),
    'xgboost':
        xgb.XGBClassifier(
            objective='binary:logistic',
            booster='gbtree',
            learning_rate=0.1,
            max_depth=6,
            min_child_weight=12,
            n_estimators=100,
            subsample=0.95
        ),
    'simple-NN': get_keras_simple_nn()
}


def get_baselines_performance(df_train, df_val, label_col='Binary', use_only=None):
    df_train = get_rdkit_features(df_train)
    df_val = get_rdkit_features(df_val)
    input_cols = [
        'BalabanJ', 'BertzCT', 'MaxAbsPartialCharge', 'MolLogP', 'MolWt', 'NumAliphaticCarbocycles',
        'NumRotatableBonds', 'RingCount', 'SlogP_VSA10', 'TPSA'
    ]
    if use_only is None:
        use_only = baselines_map.keys()
    metrics = {}
    for name, model in baselines_map.items():
        if name in use_only:
            if name == 'simple-NN':
                model.fit(df_train[input_cols].values, df_train[label_col].values, epochs=30, batch_size=32)
            else:
                model.fit(df_train[input_cols].values, df_train[label_col].values)

            y_pred = model.predict(df_val[input_cols].values).squeeze()
            y_true = df_val[label_col].values.squeeze()
            metrics[name] = calculate_metrics(y_true, y_pred)

    return pd.DataFrame(metrics).T

In [3]:
target_1 = 'p38'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)
    
target_2 = 'akt1'
base_path_2 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_2 = base_path_2+f'/data/{target_2}/data.csv'
df_akt1 = pd.read_csv(data_fpath_2).set_index('biolab_index')

with open(base_path_2+f'/data/{target_2}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_akt1 = dill.load(in_f)
with open(base_path_2+f'/data/{target_2}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_akt1 = dill.load(in_f)
    
target_3 = 'pi3k'
base_path_3 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_3 = base_path_3+f'/data/{target_3}/data.csv'
df_pi3k = pd.read_csv(data_fpath_3).set_index('biolab_index')

with open(base_path_3+f'/data/{target_3}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_pi3k = dill.load(in_f)
with open(base_path_3+f'/data/{target_3}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_pi3k = dill.load(in_f)

In [4]:
train_p38 = df_p38.loc[train_test_folds_p38[0]]            
val_p38 = df_p38.loc[train_test_folds_p38[1]]
                   

train_akt1 = df_akt1.loc[train_test_folds_akt1[0]]           
val_akt1 = df_akt1.loc[train_test_folds_akt1[1]]
                   

train_pi3k = df_pi3k.loc[train_test_folds_pi3k[0]]        
val_pi3k = df_pi3k.loc[train_test_folds_pi3k[1]]

In [5]:
#Random splits with sklearn (on our test set)
df_p38 = df_p38.reset_index(drop=True)
X_train_p38, X_val_p38, Y_train_p38, Y_val_p38 = train_test_split(df_p38.rdkit,
                                                                  df_p38.Binary,
                                                                  test_size = 0.15,
                                                                  train_size = 0.85,
                                                                  shuffle = True)
random_train_p38 = pd.DataFrame(X_train_p38)
random_train_p38['Binary'] = Y_train_p38
random_val_p38 = pd.DataFrame(X_val_p38)
random_val_p38['Binary'] = Y_val_p38
del X_train_p38,X_val_p38,Y_train_p38,Y_val_p38


df_akt1 = df_akt1.reset_index(drop=True)
X_train_akt1, X_val_akt1, Y_train_akt1, Y_val_akt1 = train_test_split(df_akt1.rdkit,
                                                                     df_akt1.Binary,
                                                                     test_size = 0.15,
                                                                     train_size = 0.85,
                                                                     shuffle = True)
random_train_akt1 = pd.DataFrame(X_train_akt1)
random_train_akt1['Binary'] = Y_train_akt1
random_val_akt1 = pd.DataFrame(X_val_akt1)
random_val_akt1['Binary'] = Y_val_akt1
del X_train_akt1,X_val_akt1,Y_train_akt1,Y_val_akt1


df_pi3k = df_pi3k.reset_index(drop=True)
X_train_pi3k, X_val_pi3k, Y_train_pi3k, Y_val_pi3k = train_test_split(df_pi3k.rdkit,
                                                                      df_pi3k.Binary,
                                                                      test_size = 0.15,
                                                                      train_size = 0.85,
                                                                      shuffle = True)
random_train_pi3k = pd.DataFrame(X_train_pi3k)
random_train_pi3k['Binary'] = Y_train_pi3k
random_val_pi3k = pd.DataFrame(X_val_pi3k)
random_val_pi3k['Binary'] = Y_val_pi3k
del X_train_pi3k,X_val_pi3k,Y_train_pi3k,Y_val_pi3k

training_p38 = [train_p38,random_train_p38]
validation_p38 = [val_p38,random_val_p38]

training_akt1 = [train_akt1,random_train_akt1]
validation_akt1 = [val_akt1,random_val_akt1]

training_pi3k = [train_pi3k,random_train_pi3k]
validation_pi3k = [val_pi3k,random_val_pi3k]

In [6]:
metrics_p38 = list()
metrics_akt1 = list()
metrics_pi3k = list()
for i in range(len(training_pi3k)):
    metrics_p38.append(get_baselines_performance(training_p38[i],validation_p38[i]))
    metrics_akt1.append(get_baselines_performance(training_akt1[i],validation_akt1[i]))
    metrics_pi3k.append(get_baselines_performance(training_pi3k[i],validation_pi3k[i]))

100%|██████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 127425.49it/s]

100%|███████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 87372.38it/s]

100%|██████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 203435.38it/s]

 21%|████████████████▏                                                            | 639/3050 [00:00<00:00, 6341.64it/s]


100%|██████████████████████████████████████████████████████████████████████████| 3050/3050 [00:00<00:00, 758576.09it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 256229.08it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 84580.67it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 255157.25it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 7501.65it/s]

100%|████████████████████████████████████████████████████████████████████████████| 509/509 [00:00<00:00, 495451.55it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 161.09it/s]
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
S

Epoch 1/30
3050/3050 [==============================] - 2s 493us/step - loss: 7.8424 - acc: 0.5134
Epoch 2/30
3050/3050 [==============================] - 0s 138us/step - loss: 7.8424 - acc: 0.5134
Epoch 3/30
3050/3050 [==============================] - 0s 138us/step - loss: 7.8424 - acc: 0.5134
Epoch 4/30
3050/3050 [==============================] - 0s 136us/step - loss: 7.8424 - acc: 0.5134
Epoch 5/30
3050/3050 [==============================] - 0s 137us/step - loss: 7.8424 - acc: 0.5134
Epoch 6/30
3050/3050 [==============================] - 0s 133us/step - loss: 7.8424 - acc: 0.5134
Epoch 7/30
3050/3050 [==============================] - 0s 132us/step - loss: 7.8424 - acc: 0.5134
Epoch 8/30
3050/3050 [==============================] - 0s 145us/step - loss: 7.8424 - acc: 0.5134
Epoch 9/30
3050/3050 [==============================] - 0s 139us/step - loss: 7.8424 - acc: 0.5134
Epoch 10/30
3050/3050 [==============================] - 0s 139us/step - loss: 7.8424 - acc: 0.5134
Epoch 11/

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████████████████████████████████████████████████████████████████████| 1834/1834 [00:00<00:00, 306480.48it/s]

100%|███████████████████████████████████████████████████████████████████████████| 1834/1834 [00:00<00:00, 79952.95it/s]

100%|██████████████████████████████████████████████████████████████████████████| 1834/1834 [00:00<00:00, 229869.52it/s]

100%|██████████████████████████████████████████████████████████████████████████| 1834/1834 [00:00<00:00, 613034.23it/s]

100%|████████████████████████████████████████████████████████████████████████████| 1834/1834 [00:01<00:00, 1612.09it/s]


100%|████████████████████████████████████████████████████████████████████████████| 306/306 [00:00<00:00, 153395.13it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 306/306 [00:00<00:00, 4092.97it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 306/306 [00:00<00:00, 3163.09it/s]

100%|████████████████████████████████████████████████████████████████████████████| 306/306 [00:00<00:00, 153450.15it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 306/306 [00:00<00:00, 6818.23it/s]

100%|████████████████████████████████████████████████████████████████████████████| 306/306 [00:00<00:00, 306533.80it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 256.16it/s]
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
S

...

Epoch 1/30
1819/1819 [==============================] - 0s 157us/step - loss: 6.6369 - acc: 0.5882
Epoch 2/30
1819/1819 [==============================] - 0s 137us/step - loss: 6.6369 - acc: 0.5882
Epoch 3/30
1819/1819 [==============================] - 0s 129us/step - loss: 6.6369 - acc: 0.5882
Epoch 4/30
1819/1819 [==============================] - 0s 146us/step - loss: 6.6369 - acc: 0.5882
Epoch 5/30
1819/1819 [==============================] - 0s 134us/step - loss: 6.6369 - acc: 0.5882
Epoch 6/30
1819/1819 [==============================] - 0s 135us/step - loss: 6.6369 - acc: 0.5882
Epoch 7/30
1819/1819 [==============================] - 0s 134us/step - loss: 6.6369 - acc: 0.5882
Epoch 8/30
1819/1819 [==============================] - 0s 137us/step - loss: 6.6369 - acc: 0.5882
Epoch 9/30
1819/1819 [==============================] - 0s 145us/step - loss: 6.6369 - acc: 0.5882
Epoch 10/30
1819/1819 [==============================] - 0s 135us/step - loss: 6.6369 - acc: 0.5882
Epoch 11/

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████████████████████████████████████████████████████████████████████| 3190/3190 [00:00<00:00, 266562.33it/s]

 46%|███████████████████████████████████▏                                        | 1477/3190 [00:00<00:00, 2926.20it/s]


100%|███████████████████████████████████████████████████████████████████████████| 3190/3190 [00:00<00:00, 84171.58it/s]

100%|██████████████████████████████████████████████████████████████████████████| 3190/3190 [00:00<00:00, 213254.97it/s]

100%|██████████████████████████████████████████████████████████████████████████| 3190/3190 [00:00<00:00, 456930.19it/s]

100%|████████████████████████████████████████████████████████████████████████████| 564/564 [00:00<00:00, 282525.67it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 564/564 [00:00<00:00, 94272.80it/s]

100%|████████████████████████████████████████████████████████████████████████████| 564/564 [00:00<00:00, 113083.20it/s]

100%|████████████████████████████████████████████████████████████████████████████| 564/564 [00:00<00:00, 282762.07it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 121.82it/s]
C:\Users\tomas\miniconda3\envs\bi

Epoch 1/30
3190/3190 [==============================] - 0s 151us/step - loss: 5.8510 - acc: 0.6370
Epoch 2/30
3190/3190 [==============================] - 0s 133us/step - loss: 5.8510 - acc: 0.6370
Epoch 3/30
3190/3190 [==============================] - 0s 134us/step - loss: 5.8510 - acc: 0.6370
Epoch 4/30
3190/3190 [==============================] - 0s 136us/step - loss: 5.8510 - acc: 0.6370
Epoch 5/30
3190/3190 [==============================] - 0s 138us/step - loss: 5.8510 - acc: 0.6370
Epoch 6/30
3190/3190 [==============================] - 0s 143us/step - loss: 5.8510 - acc: 0.6370
Epoch 7/30
3190/3190 [==============================] - 0s 137us/step - loss: 5.8510 - acc: 0.6370
Epoch 8/30
3190/3190 [==============================] - 0s 144us/step - loss: 5.8510 - acc: 0.6370
Epoch 9/30
3190/3190 [==============================] - 0s 144us/step - loss: 5.8510 - acc: 0.6370
Epoch 10/30
3190/3190 [==============================] - 0s 143us/step - loss: 5.8510 - acc: 0.6370
Epoch 11/

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
pd.DataFrame(metrics_p38[0]).to_csv("../../../../Desktop/binding/thesis english/Results/5-Baselines/test_p38.csv")
pd.DataFrame(metrics_p38[1]).to_csv("../../../../Desktop/binding/thesis english/Results/5-Baselines/random_p38.csv")

In [9]:
pd.DataFrame(metrics_akt1[0]).to_csv("../../../../Desktop/binding/thesis english/Results/5-Baselines/test_akt1.csv")
pd.DataFrame(metrics_akt1[1]).to_csv("../../../../Desktop/binding/thesis english/Results/5-Baselines/random_akt1.csv")

In [10]:
pd.DataFrame(metrics_pi3k[0]).to_csv("../../../../Desktop/binding/thesis english/Results/5-Baselines/test_pi3k.csv")
pd.DataFrame(metrics_pi3k[1]).to_csv("../../../../Desktop/binding/thesis english/Results/5-Baselines/random_pi3k.csv")